In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import random
import os

np.random.seed(1)
random.seed(1)
DATA_DIR = "/home/vuthede/AI/zalo/zalo-hit-song-prediction/csv/"
TRAININFO = os.path.join(DATA_DIR, "train_info.tsv")
TRAINRANK =  os.path.join(DATA_DIR, "train_rank.csv")
TESTINFO = os.path.join(DATA_DIR, "test_info.tsv")
SUBMISSION = os.path.join(DATA_DIR, "submission.csv")

# Prepare data
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(os.path.join(DATA_DIR, "all_track_info.csv"))
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(os.path.join(DATA_DIR, "all_track_audio_features.csv"))
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df_album_hash = pd.read_csv(os.path.join(DATA_DIR, "album_hash.csv")) 
df =df.merge(df_album_hash,left_on="ID",right_on="ID", how="left")

# Sort by ID
df = df.sort_values(by=['ID'])
df= df.reset_index()

df.head(2)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,index,ID,title,artist_name,artist_id,composers_name,composers_id,release_time,label,dataset,...,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale,Unnamed: 0_x,album_hash_x,Unnamed: 0_y,album_hash_y
0,0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,551,Châu Đình An,5765,2017-10-01 22:07:00,7.0,train,...,G,major,G,major,G,major,0,3c5c7e72f9112800,0,3c5c7e72f9112800
1,1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,455,Trường Sa,100105,2017-10-01 20:58:00,3.0,train,...,C,minor,C,minor,C,minor,1,f39df9c10843e3fc,1,f39df9c10843e3fc


In [3]:
from format_features import format_features, add_artist_bestrank
df = format_features(df)


There is 0.24038838760298156 ratio is nan album
There is 0.0017653981953707335 ratio is nan genre
There is 0.24038838760298156 ratio is nan album_artist
There is 0.0007846214201647705 ratio is nan track
There is 0.6722244017261672 ratio is nan lyric
2711 raw titles are identical between songs: 7485 unique titles
After cleaning brackets etc. only 6202 unique titles remain, i.e. 1283 are highly similar titles 
In making albumHashandName, we filled in: 146 values Of the total albumHashAndName 171 nan
In making albumHashAndNameAndReleaseday, we filled in the: 171 values remaining using the release second hash
There is a statistically signficiant relationship between English-like title and rank. So adding feature: isEnglishLikeTitle


In [ ]:
all_features_in_order = {## album is Redundant feature use: albumHashAndNameAndReleaseday", # album name from mp3 metadata textual
                          "albumHashAndNameAndReleaseday":"category",
                          "len_album_name":"int64",
                          "isRemixAlbum":"category",
                          "isOSTAlbum":"category",
                          "isSingleAlbum":"category",
                          "isBeatAlbum":"category",
                          "isTopHitAlbum":"category",
                          "isCoverAlbum":"category",
                          "isEPAlbum":"category",
                          "isLienKhucAlbum":"category",
                          "album_name_is_title_name":"category",
                          "artist_name":"category",
                          "composers_name":"category",
                          "copyright":"category" ,
                          "artist_id_min":"category",
                          "artist_id_max":"category", 
                          "composers_id_min":"category", 
                          "composers_id_max":"category",
                          "genre":"category", 
                          "album_artist":"category", # album artist name
                          "album_artist_contain_artistname":"category",
                          "track":"float64", # float between 0 and 1 representing track_num/total_tracks
                          "istrack11":"category", # 1 if first track
                          # "lyric":"string" # Not a trainable feature
                          "islyric":"category",
                          "num_line_lyric":"int64",
                          "no_artist":"int64",
                          "no_composer":"int64",
                          #"datetime":"datetime64", # Not a trainable feature
                          "day":"category",
                          "month":"category",
                          "year":"category",
                          "hour":"category",
                          "dayofyear":"category",
                          "weekday":"category",
                          "isHoliday":"category",
                          "len_of_songname":"int64",
                          "isRemix":"category",
                          "isOST":"category",
                          "isBeat":"category",
                          "isVersion":"category",
                          "isCover":"category",
                          "isLienKhuc":"category",
                          "day_release":"int64", # the specific day of the day across all days (> 365)  
                          "datetimeIs_month_end":"category",
                          "datetimeIs_month_start":"category",
                          "datetimeIs_quarter_end":"category",
                          "datetimeIs_quarter_start":"category",
                          "datetimeIs_year_end":"category",
                          "datetimeIs_year_start":"category",
                          "datetimeDayofweek":"category",
                          "tonal.chords_key":"category",
                          "tonal.chords_scale":"category",
                          'datetimeweekday_cos':"float64",
                          'datetimeweekday_sin':"float64", 
                          'datetimeday_month_cos':"float64",
                          'datetimeday_month_sin':"float64", 
                          'datetimemonth_year_cos':"float64",
                          'datetimemonth_year_sin':"float64", 
                          'datetimeday_year_cos':"float64",
                          'datetimeday_year_sin':"float64",
                          "length":"float64", # length of the song (s?)
                          # (title feature likely rendered redundant - use title_truncated )"title":"category",
                          'title_truncated':"category",
                          ###########################
                          # Warning: the below features those that require "global" knowledge beyond that example
                          ###########################
                          "numsongInAlbum":"category",
                          "isSingleAlbum_onesong":"category",
                           #"num_song_released_that_week":'int64',
                          "num_song_release_in_final_month":"int64",  
                          "freq_artist":"int64",  # number of times the unique artist string is present in dataset
                          "freq_artist_min":"int64", # number of times the first listed artist is present in dataset
                          "num_album_per_min_artist":"int64",
                          "num_album_per_min_composer":"int64",
                          "freq_composer_min":"int64",   
                          "isEnglishLikeTitle":"category" # has some words with no special charachters - in practice selects short and weird titles
                          }






def add_artist_bestrank(df,all_features_in_order):
    from datetime import datetime
    import re

    def split_id(s):
        return re.split(',|\.',s)

    m = df.artist_id.unique()
    idx_lst = []
    for idx in m:
        ps = split_id(idx)
        for i in ps:
            idx_lst.append(i)

    id_lst = list(set(idx_lst))

    def condition(df, artist_id):
        r = df.artist_id.apply(lambda x: artist_id in split_id(x))
        return r

    df_train = df[df.dataset=="train"]
    data= [df_train[condition(df_train, artist_id)].label.agg(["mean","std","count"]) for artist_id in id_lst]
    new_df = pd.DataFrame(data=data)
    new_df["artist_id"] =  id_lst

    new_df.dropna(inplace=True)
    new_df.set_index('artist_id', inplace=True)
    art_dict = new_df.to_dict()

    def best_count_id(values):
        ids = split_id(values)
        temp_mean = 0
        temp_id = str(min([int(a) for a in ids]))
        for id in ids:
            try:
                if art_dict['count'][id] > temp_mean:
                    temp_mean = art_dict['count'][id]
                    temp_id = id
            except:
                temp_mean = temp_mean
                temp_id = temp_id
        return temp_id

    df['artist_bestcount'] = df['artist_id'].apply(best_count_id)

    def best_mean_id(values):
        ids = split_id(values)
        temp_mean = 10
        temp_id = str(min([int(a) for a in ids]))
        for id in ids:
            try:
                if art_dict['mean'][id] < temp_mean:
                    temp_mean = art_dict['mean'][id]
                    temp_id = id
            except:
                temp_mean = temp_mean
                temp_id = temp_id
        return temp_id

    df['artist_bestmeanrank'] = df['artist_id'].apply(best_mean_id)

    def best_std_id(values):
        ids = split_id(values)
        temp_std = 10
        temp_id = str(min([int(a) for a in ids]))
        for id in ids:
            try:
                if art_dict['std'][id] < temp_std:
                    temp_std = art_dict['std'][id]
                    temp_id = id
            except:
                temp_std = temp_std
                temp_id = temp_id
        return temp_id

    df['artist_beststdrank'] = df['artist_id'].apply(best_std_id)

    df['artist_mean_id'] = df['artist_mean_id']
    df['artist_std_id'] = df['artist_std_id']
    df['artist_bestcount'] = df['artist_bestcount']

def add_raw_audio_features(df, all_features_in_order):
    not_included_columns = set(df.columns) - set([i for i in all_features_in_order.keys()]) - set(["tonal.chords_key", "tonal.chords_scale"])
    audio_feature_names = sorted([i for i in not_included_columns if i.split(".")[0] == 'tonal'
            or  i.split(".")[0] == 'rhythm'
            or  i.split(".")[0]=='lowlevel'])
    pd.set_option('display.max_columns', 500)
    audio_feature_names_dict = {name:"float64" for name in audio_feature_names}
    audio_feature_names_dict["tonal.chords_key"] = "category"
    audio_feature_names_dict["tonal.chords_scale"] = "category"
    audio_feature_names_dict["tonal.key_edma.key"] = "category"
    audio_feature_names_dict["tonal.key_edma.scale"] = "category"
    audio_feature_names_dict["tonal.key_krumhansl.key"] = "category"
    audio_feature_names_dict["tonal.key_krumhansl.scale"] = "category"
    audio_feature_names_dict["tonal.key_temperley.key"] = "category"
    audio_feature_names_dict["tonal.key_temperley.scale"] = "category"
    all_features_in_order.update(audio_feature_names_dict)
    return all_features_in_order

all_features_in_order = add_raw_audio_features(df, all_features_in_order)
all_features_in_order_list = list(all_features_in_order.keys())
for feat_name, feat_type in all_features_in_order.items():
    try:
        df[feat_name] = df[feat_name].astype(feat_type)
    except ValueError:
        print(feat_name, feat_type)
        raise


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from format_features import baysianEncodeFeature


best_chosen_features = 
["albumHashAndNameAndReleaseday_enc", "istrack11", "no_artist", "no_composer","freq_artist", "freq_composer",
 "year", "month","hour", "day", "len_of_songname", 
"isRemix", "isOST", "isBeat", "isVersion", "isCover",  "num_song_release_in_final_month",
"length", "genre", "track","album_artist", "islyric", "album_artist_contain_artistname",
"len_album_name", "isRemixAlbum", "isOSTAlbum", "isSingleAlbum", "album_name_is_title_name",
"isBeatAlbum", "isCoverAlbum", "artist_name_cat","composers_name_cat","copyright_cat" ,
# "artist_id_min_cat", "composers_id_min_cat",  "artist_id_max_cat", "composers_id_max_cat", 
"freq_artist_min", "freq_composer_min","dayofyear","weekday","isHoliday",
"num_album_per_min_artist", "num_album_per_min_composer", 
"numsongInAlbum","isSingleAlbum_onesong",
 "artist_mean_id", "artist_std_id" ,"artist_count_id","title_cat","num_same_title"]


df = df.copy()

categorical_col_names = df[all_features_in_order_list].select_dtypes(include=['category']).columns
for colname in categorical_col_names:
    df[colname] = df[colname].cat.codes
    






chosen_features = bens_chosen_features

df_train = df[df.dataset=="train"]
df_test = df[df.dataset=="test"]

param = {
    'bagging_freq': 20,          
    'bagging_fraction': 0.95,   'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 0.1,     'learning_rate': 0.001,
    'max_depth': -1,             'metric':'root_mean_squared_error', 'min_data_in_leaf': 100, # cjha   
       'num_leaves': 20,            
    'num_threads': 4,              'tree_learner': 'serial',   'objective': 'regression',
    'reg_alpha': 0.1002650970728192, 'reg_lambda': 0.1003427518866501,'verbosity': 1,
    "seed": 99999
}

folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=99999)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
labels= df_train.label
# fig, axes = plt.subplots(5, 1, figsize=(10, 10*5))
# axes = axes.flat
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train.label.values)):
    print("Fold {}".format(fold_))
    
    df_train = baysianEncodeFeature(df_train, trn_idx, 'albumHashAndNameAndReleaseday', prior_weight=4, fillmissing=-1, suffix='_enc')
    df_train = baysianEncodeFeature(df_train, trn_idx, 'artist_id_min', prior_weight=4, fillmissing=-1, suffix='_enc')
    df_train = baysianEncodeFeature(df_train, trn_idx, 'title_truncated', prior_weight=4, fillmissing=-1, suffix='_enc')

    train_df_fold = df_train.iloc[trn_idx][chosen_features]
    val_df_fold = df_train.iloc[val_idx][chosen_features]
    trn_data = lgb.Dataset(train_df_fold, label=labels.iloc[trn_idx])
    val_data = lgb.Dataset(val_df_fold, label=labels.iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 20000)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][chosen_features], num_iteration=clf.best_iteration)
    predictions += clf.predict(df_test[chosen_features], num_iteration=clf.best_iteration) / folds.n_splits
# training's rmse: 0.428706	valid_1's rmse: 1.57874


In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

print("RMSE: {:<8.5f}".format(sqrt(mean_squared_error(df_train.label, oof))))
sub = pd.DataFrame({"ID": df_test.ID.values})
sub["label"] = predictions
sub.to_csv("submission_lightgbm.csv", index=False, header=False)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30,30))
lgb.plot_importance(clf, max_num_features=20,importance_type='gain')
